# Vehicle routing with higher order penalties

This notebook showcases the results of the eponymous bachelor thesis.
## Contents
* [Setup](#setup)
* [Routing Problem](#routing_problem)
* [Lane-Level Topology approach](#lane_level)
* [X-Graph approach](#x_graph)
* [Comparison](#x_graph)

## Setup <a id='setup'></a>

In [ ]:
import json
import os
from io import StringIO
from typing import Optional, List

import matplotlib.pyplot as plt
import pandas as pd
from dotenv import load_dotenv
from geojson import FeatureCollection
from mapboxgl.utils import create_color_stops
from mapboxgl.viz import LinestringViz
from ortools.constraint_solver.routing_enums_pb2 import LocalSearchMetaheuristic, FirstSolutionStrategy

from src.batch_runner.batch_runner import batch_run
from src.config.map import satellite_style
from src.helpers.helpers import format_matrix
from src.optimizer.ebg_optimizer import EBGOptimizer
from src.optimizer.x_graph_optimizer import XGraphOptimizer
from src.osrm.interface import OSRMInterface
from src.routing_problem.connections.complete import XGraphNode, FirstXGraphNode, SelfXGraphNode, LastXGraphNode
from src.routing_problem.connections.lanelet import LaneletConnection
from src.routing_problem.connections.route import RouteConnection
from src.routing_problem.creator.creator import create_routing_problem
from src.routing_problem.lanelet import Lanelet, FirstLanelet, LastLanelet
from src.routing_problem.maneuver.maneuver_type import ManeuverType
from src.routing_problem.maneuver.modifier import ManeuverModifier
from src.routing_problem.routing_problem import RoutingProblem

In [ ]:
% load_ext autoreload
% autoreload 2

You need to create a [free Mapbox account](https://account.mapbox.com/auth/signup/) and add your access token to the .env file in the root of the repository.

In [ ]:
load_dotenv()
token = os.getenv('MAPBOX_ACCESS_TOKEN')

Change the routing problem name here to load another routing problem.

In [ ]:
rp_name = "stuttgart_small"
rp_map_data = "Stuttgart/stuttgart-regbez-latest.osrm"  # OSRM path to the underlying map data
rp_path = f"data/{rp_name}.osm"

## Routing problem <a id='routing_problem'></a>

Load routing problem and show it on the interactive map. Hover the cursor above lanelets to see additional information.

In [ ]:
rp: RoutingProblem = create_routing_problem(rp_path)
rp.visualize()

Shorten lanelets for better visibility

In [ ]:
rp.shorten_lanelets()
rp.visualize(zoom=14)

## Lane-Level Topology Approach <a id='lane_level'></a>

Add lane-level topology by applying heuristics described in the thesis.

In [ ]:
connections = []
# First pass
for segment in rp.segments:
    for previous_segment in segment.previous_segments:
        maneuver = rp.maneuvers[(previous_segment.id, segment.id)]
        # Check if maneuver is left, right and/or merge
        left = maneuver.modifier in [ManeuverModifier.Left, ManeuverModifier.SlightLeft,
                                     ManeuverModifier.SharpLeft, ManeuverModifier.UTurn]
        right = maneuver.modifier in [ManeuverModifier.Right, ManeuverModifier.SlightRight,
                                      ManeuverModifier.SharpRight]
        merge = maneuver.type == ManeuverType.Merge
        # Left
        if left and not merge or right and merge:
            for i, lanelet in enumerate(segment.lanelets):
                if i < len(previous_segment.lanelets):
                    connections.append(LaneletConnection(previous_segment.lanelets[i],
                                                         lanelet,
                                                         maneuver))
        # Right
        elif right and not merge or left and merge:
            for i, lanelet in enumerate(reversed(segment.lanelets)):
                if i < len(previous_segment.lanelets):
                    connections.append(LaneletConnection(previous_segment.lanelets[-(i + 1)],
                                                         lanelet,
                                                         maneuver))

# Second pass
for segment in rp.segments:
    for previous_segment in segment.previous_segments:
        maneuver = rp.maneuvers[(previous_segment.id, segment.id)]
        # Skip straight maneuvers
        if maneuver.modifier != ManeuverModifier.Straight:
            continue

        # Count free lanelets (lanelets without incoming connections)
        lanelets_to = len(segment.lanelets)
        lanelets_from = len(previous_segment.lanelets)
        free_lanelets_to = list(filter(lambda x: not x.has_incoming_connection, segment.lanelets))
        free_lanelets_from = list(filter(lambda x: not x.has_outgoing_connection, previous_segment.lanelets))
        free_lanelets_to_count = len(free_lanelets_to)
        free_lanelets_from_count = len(free_lanelets_from)

        if lanelets_to == lanelets_from:
            for i in range(lanelets_to):
                connections.append(LaneletConnection(previous_segment.lanelets[i],
                                                     segment.lanelets[i],
                                                     maneuver))
        elif free_lanelets_to_count == lanelets_from:
            for i in range(free_lanelets_to_count):
                connections.append(LaneletConnection(previous_segment.lanelets[i],
                                                     free_lanelets_to[i],
                                                     maneuver))
        elif lanelets_to == free_lanelets_from_count:
            for i in range(lanelets_to):
                connections.append(LaneletConnection(free_lanelets_from[i],
                                                     segment.lanelets[i],
                                                     maneuver))
        elif lanelets_to > free_lanelets_from_count:
            for i in range(lanelets_to):
                connections.append(LaneletConnection(free_lanelets_from[min(i, free_lanelets_from_count - 1)],
                                                     segment.lanelets[i],
                                                     maneuver))
        elif lanelets_to < free_lanelets_from_count:
            for i in range(free_lanelets_from_count):
                connections.append(LaneletConnection(free_lanelets_from[i],
                                                     segment.lanelets[min(i, lanelets_to - 1)],
                                                     maneuver))

# Convert stuff to geojson
rp_json = rp.to_json()
connections_json = [connection.to_json() for connection in connections]
geojson = FeatureCollection(features=rp_json + connections_json)

# Show everything on map
color_breaks = [0, 1]
color_stops = create_color_stops(color_breaks, colors=["#3ad21b", "#ff0505"])
viz = LinestringViz(data=geojson,
                    color_property="type",
                    color_stops=color_stops,
                    line_width_default='2',
                    center=(9.092, 48.731),
                    zoom=18,
                    style=satellite_style)
viz.show()

Load RP's distance/duration matrix. If it doesn't exist, request it from Atlatec's OSRM fork.

In [ ]:
matrix_path = f"data/{rp_name}_matrix.json"

if os.path.exists(matrix_path):
    with open(matrix_path) as f:
        matrix = json.load(matrix_path)
else:
    sources = [int(segment.id) for segment in rp.segments]
    destinations = sources
    try:
        durations_matrix = OSRMInterface.request_table(sources, destinations, rp_map_data)[1]
    except Exception:
        print("Couldn't get the durations matrix from OSRM.")
    else:
        matrix = format_matrix(sources, destinations, durations_matrix)

        with open(matrix_path, "w") as f:
            json.dump(matrix, f)
        print(f"Dumped {len(matrix)} x {len(matrix)} durations matrix: {matrix_path}")

In [ ]:
# Create connections. First for left and right turns, then for forward.
connections = []
for segment in rp.segments:
    for previous_segment in segment.previous_segments:
        maneuver = rp.maneuvers[(previous_segment.id, segment.id)]
        left = maneuver.modifier in [ManeuverModifier.Left, ManeuverModifier.SlightLeft,
                                     ManeuverModifier.SharpLeft, ManeuverModifier.UTurn]
        right = maneuver.modifier in [ManeuverModifier.Right, ManeuverModifier.SlightRight,
                                      ManeuverModifier.SharpRight]
        merge = maneuver.type == ManeuverType.Merge
        # Left
        if left and not merge or right and merge:
            for i, lanelet in enumerate(segment.lanelets):
                if i < len(previous_segment.lanelets):
                    connections.append(LaneletConnection(previous_segment.lanelets[i],
                                                         lanelet,
                                                         maneuver))
        # Right
        elif right and not merge or left and merge:
            for i, lanelet in enumerate(reversed(segment.lanelets)):
                if i < len(previous_segment.lanelets):
                    connections.append(LaneletConnection(previous_segment.lanelets[-(i + 1)],
                                                         lanelet,
                                                         maneuver))

# Now connect straights
for segment in rp.segments:
    for previous_segment in segment.previous_segments:
        maneuver = rp.maneuvers[(previous_segment.id, segment.id)]
        if maneuver.modifier != ManeuverModifier.Straight:
            continue

        lanelets_to = len(segment.lanelets)
        lanelets_from = len(previous_segment.lanelets)
        free_lanelets_to = list(filter(lambda x: not x.has_incoming_connection, segment.lanelets))
        free_lanelets_from = list(filter(lambda x: not x.has_outgoing_connection, previous_segment.lanelets))
        free_lanelets_to_count = len(free_lanelets_to)
        free_lanelets_from_count = len(free_lanelets_from)

        if lanelets_to == lanelets_from:
            for i in range(lanelets_to):
                connections.append(LaneletConnection(previous_segment.lanelets[i],
                                                     segment.lanelets[i],
                                                     maneuver))
        elif free_lanelets_to_count == lanelets_from:
            for i in range(free_lanelets_to_count):
                connections.append(LaneletConnection(previous_segment.lanelets[i],
                                                     free_lanelets_to[i],
                                                     maneuver))
        elif lanelets_to == free_lanelets_from_count:
            for i in range(lanelets_to):
                connections.append(LaneletConnection(free_lanelets_from[i],
                                                     segment.lanelets[i],
                                                     maneuver))
        elif lanelets_to > free_lanelets_from_count:
            for i in range(lanelets_to):
                connections.append(LaneletConnection(free_lanelets_from[min(i, free_lanelets_from_count - 1)],
                                                     segment.lanelets[i],
                                                     maneuver))
        elif lanelets_to < free_lanelets_from_count:
            for i in range(free_lanelets_from_count):
                connections.append(LaneletConnection(free_lanelets_from[i],
                                                     segment.lanelets[min(i, lanelets_to - 1)],
                                                     maneuver))

# Convert stuff to geojson
rp_json = rp.to_json()
connections_json = [connection.to_json() for connection in connections]
geojson = FeatureCollection(features=rp_json + connections_json)

# Show everything on map
color_breaks = [0, 1]
color_stops = create_color_stops(color_breaks, colors=["#3ad21b", "#ff0505"])
viz = LinestringViz(data=geojson,
                    color_property="type",
                    color_stops=color_stops,
                    line_width_default='2',
                    center=(9.092, 48.731),
                    zoom=18,
                    style=satellite_style)
viz.show()

Solve Lane Topology TSP

In [ ]:
lanelet_connections = {(connection.lanelet_from, connection.lanelet_to) for connection in connections}
optimiser = EBGOptimizer(nodes=[FirstLanelet()] + rp.lanelets + [LastLanelet()],
                         matrix=matrix,
                         local_search_metaheuristic=LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH,
                         first_solution_strategy=FirstSolutionStrategy.AUTOMATIC,
                         max_optimisation_duration=2,
                         connections=lanelet_connections,
                         check_topology=True)
optimal_order, optimization_history = optimiser.optimize()
print(f"Optimisation history: {optimization_history}")

Display the route

In [ ]:
route_connections = []
previous_connection: Optional[Lanelet] = None
for lanelet in optimal_order[1:-1]:
    if previous_connection is not None:
        cost = previous_connection.get_cost_to(lanelet,
                                               matrix,
                                               lanelet_connections)
        # if cost < 20:
        route_connections.append(RouteConnection(previous_connection,
                                                 lanelet,
                                                 cost,
                                                 1))
    previous_connection = lanelet

# Convert stuff to geojson
rp_json = rp.to_json()
connections_json = [connection.to_json() for connection in route_connections]
geojson = FeatureCollection(features=rp_json + connections_json)

# Show everything on map
color_breaks = [0, 1]
color_stops = create_color_stops(color_breaks, colors=["#3ad21b", "#ff0505"])
viz = LinestringViz(data=geojson,
                    color_property="type",
                    color_stops=color_stops,
                    line_width_default='2',
                    center=(9.092, 48.731),
                    zoom=18,
                    style=satellite_style)
viz.show()

## X-Graph approach <a id='x_graph'></a>

Create X-Graph from routing problem

In [ ]:
x_nodes: List[XGraphNode] = [FirstXGraphNode()]
disjunctions: List[List[int]] = []

for lanelet_to in rp.lanelets:
    disjunctions.append([])
    segment_to = lanelet_to.segment

    for lanelet_from in rp.lanelets:
        segment_from = lanelet_from.segment

        if (segment_from.id, segment_to.id) in rp.maneuvers:
            maneuver = rp.maneuvers[(segment_from.id, segment_to.id)]

            x_nodes.append(XGraphNode(lanelet_from=lanelet_from,
                                      lanelet_to=lanelet_to,
                                      maneuver=maneuver))
            disjunctions[-1].append(len(x_nodes) - 1)

    # If there are no incoming maneuvers to the passlet (lanelet is a passlet i X-Graph context),
    # create a SelfXGraphNode to include the passlet in the routing result. SelfXGraphNode represents
    # a passlet instead of a maneuver.
    if len(disjunctions[-1]) == 0:
        disjunctions = disjunctions[:-1]
        x_nodes.append(SelfXGraphNode(lanelet_to))

x_nodes.append(LastXGraphNode())

Count number of nodes in different RP representations and other stats.

In [ ]:
print(f"Number of nodes in NBG: {rp.nbg_nodes_number}")
print(f"Number of nodes in EBG (our definition with lanelets as nodes): {len(rp.lanelets)}")
print(f"Number of nodes in X-Graph: {len(x_nodes)}")
print(f"Number of disjunctions: {len(disjunctions)}")
print(f"Number of segments: {len(rp.segments)}")
print(f"Average branching factor (ABF): {round(rp.average_branching_factor, 2)}")

Compare average branching factor for different RP scenarios.

In [ ]:
rp_highway = create_routing_problem("data/highway.osm")
rp_mixed = create_routing_problem("data/mixed.osm")
rp_urban = create_routing_problem("data/urban.osm")

fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
scenarios = ["Highway", "Mixed", "Urban"]
ys = [rp_highway.average_branching_factor, rp_mixed.average_branching_factor, rp_urban.average_branching_factor]
ax.bar(scenarios, ys, color=["#3ad21b", "#055AFF", "#FA05FF"])
ax.set_ylabel('average branching factor')
ax.set_xlabel('scenario type')
plt.savefig("exports/abf.png", bbox_inches='tight', dpi=400)
plt.show()

Compare different local search metaheuristics

In [ ]:
heuristics = [LocalSearchMetaheuristic.GREEDY_DESCENT, LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH,
              LocalSearchMetaheuristic.SIMULATED_ANNEALING, LocalSearchMetaheuristic.TABU_SEARCH]
batch = []
for heuristic in heuristics:
    batch.append((EBGOptimizer(nodes=rp.lanelets,
                               matrix=matrix,
                               local_search_metaheuristic=heuristic,
                               first_solution_strategy=FirstSolutionStrategy.AUTOMATIC,
                               max_optimisation_duration=600,
                               check_topology=False).optimize, tuple()))
return_dict = batch_run(batch)

fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
heuristics = ["Greedy Descent", "Guided Local Search", "Simulated Annealing", "Tabu Search"]
ys = [return_dict[i]["history"].values()[-1] for i in range(len(heuristics))]
ax.bar(heuristics, ys, color=["#3ad21b", "#055AFF", "#FA05FF", "#FFE605"])
plt.xticks(rotation=15)
ax.set_ylabel('optimization result (s)')
ax.set_xlabel('local search strategy')
plt.savefig("exports/lss.png", bbox_inches='tight', dpi=400)
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.axes()
ax.set_ylabel('optimization result (s)')
ax.set_xlabel('time (s)')
colors = ["#3ad21b", "#055AFF", "#FA05FF", "#FFE605"]

for i in range(len(heuristics)):
    xs = [float(x) for x in return_dict[i]["history"].keys()]
    ys = list(return_dict[i]["history"].values())

    ax.plot(xs, ys, label=heuristics[i], color=colors[i])

plt.legend()
plt.savefig("exports/lss_comparison.png", bbox_inches='tight', dpi=400)
plt.show()

In [ ]:
batch = [(EBGOptimizer(nodes=rp.lanelets,
                       matrix=matrix,
                       local_search_metaheuristic=LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH,
                       first_solution_strategy=FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION,
                       max_optimisation_duration=600,
                       check_topology=False).optimize, tuple()),
         (EBGOptimizer(nodes=rp.lanelets,
                       matrix=matrix,
                       local_search_metaheuristic=LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH,
                       first_solution_strategy=FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION,
                       max_optimisation_duration=600,
                       check_topology=True,
                       connections=connections).optimize, tuple()),
         (XGraphOptimizer(nodes=x_nodes,
                          matrix=matrix,
                          disjunctions=disjunctions,
                          local_search_metaheuristic=LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH,
                          first_solution_strategy=FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION,
                          max_optimisation_duration=600,
                          straight_non_straight_maneuver_penalty=120,
                          non_straight_straight_maneuver_penalty=0).optimize, tuple()),
         (XGraphOptimizer(nodes=x_nodes,
                          matrix=matrix,
                          disjunctions=[],
                          local_search_metaheuristic=LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH,
                          first_solution_strategy=FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION,
                          max_optimisation_duration=600,
                          straight_non_straight_maneuver_penalty=120,
                          non_straight_straight_maneuver_penalty=0).optimize, tuple()),
         ]

return_dict = batch_run(batch, measure_resources=True)

fig = plt.figure()
ax = plt.axes()
ax.set_ylabel('optimization result (% of initial solution)')
ax.set_xlabel('time (s)')

approaches = ["Current AtlaRoute approach", "Lane topology", "X-Graph", "X-Graph w/o disjunctions"]
colors = ["#3ad21b", "#055AFF", "#FA05FF", "#FFE605"]
for i, approach in enumerate(approaches):
    xs = [float(x) for x in return_dict[i]['history'].keys()]
    ys = list(return_dict[i]['history'].values())
    max_y = max(ys)
    ys = [(y / max_y) * 100 for y in ys]

    ax.plot(xs, ys, label=approach, color=colors[i])

ax.legend(loc="center left", bbox_to_anchor=(0.45, 0.25))
plt.savefig("exports/approaches_comparison.png", bbox_inches='tight', dpi=400)
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.axes()
ax.set_ylabel('memory usage (MB)')
ax.set_xlabel('time (s)')

for i, approach in enumerate(approaches):
    memory = return_dict[i]['memory'][:600]
    xs = [i for i in range(len(memory))]
    ys = memory
    ax.plot(xs, ys, label=approaches[i], color=colors[i])

plt.legend()
plt.savefig("exports/memory_usage.png", bbox_inches='tight', dpi=400)
plt.show()

In [ ]:
strategies = [FirstSolutionStrategy.AUTOMATIC,
              FirstSolutionStrategy.PATH_CHEAPEST_ARC,
              FirstSolutionStrategy.CHRISTOFIDES,
              FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION,
              FirstSolutionStrategy.SAVINGS,
              FirstSolutionStrategy.LOCAL_CHEAPEST_INSERTION,
              FirstSolutionStrategy.FIRST_UNBOUND_MIN_VALUE,
              FirstSolutionStrategy.LOCAL_CHEAPEST_ARC,
              FirstSolutionStrategy.ALL_UNPERFORMED]
batch = []
for strategy in strategies:
    batch.append((EBGOptimizer(nodes=rp.lanelets,
                               matrix=matrix,
                               local_search_metaheuristic=LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH,
                               first_solution_strategy=FirstSolutionStrategy.AUTOMATIC,
                               max_optimisation_duration=600,
                               check_topology=False).optimize, tuple()))
return_dict = batch_run(batch)

strategies = ["automatic",
              "path cheapest arc",
              "christofides",
              "parallel cheapest insertion",
              "savings",
              "local cheapest insertion",
              "first unbound min value",
              "first unbound min value",
              "local cheapest arc",
              "all unperformed"]
# Create xml
s = "first solution strategy,performance,duration\n"
for i in range(len(strategies)):
    s.append(f"{strategies[i]},{return_dict[i]['history'].values()[0]},{return_dict[i]['history'].keys()[0]}\n")
s = StringIO(s)

# Display xml with Pandas
df = pd.read_csv(s, index_col=0, delimiter=',', skipinitialspace=True)

fig = plt.figure()  # Create matplotlib figure
ax = fig.add_subplot(111)  # Create matplotlib axes
ax2 = ax.twinx()  # Create another axes that shares the same x-axis as ax.
width = 0.4

df.performance.plot(kind='bar', color='#3ad21b', ax=ax, width=width, position=1)
df.duration.plot(kind='bar', color='#055AFF', ax=ax2, width=width, position=0)

ax.set_ylabel('first solution performance (s)')
ax.yaxis.label.set_color('#3ad21b')
ax2.set_ylabel('runtime (s)')
ax2.yaxis.label.set_color('#055AFF')

plt.savefig("exports/fss.png", bbox_inches='tight', dpi=400)
plt.show()

In [ ]:
cut_x_graph_order = order_x_graph[1:-1]
lanelets_order = []

for maneuver in cut_x_graph_order:
    lanelets_order.append(maneuver.lanelet_from)
    lanelets_order.append(maneuver.lanelet_to)

shortened_x_graph_order = []
for lanelet in lanelets_order:
    if len(shortened_x_graph_order) > 0 and shortened_x_graph_order[-1].segment.id == lanelet.segment.id:
        continue
    shortened_x_graph_order.append(lanelet)

In [ ]:
segments = [lanelet.segment for lanelet in order_normal[1:-1]]
route = OSRMInterface.request_route(segments, "Stuttgart/stuttgart-regbez-latest.osrm")
print(f"Duration: {route['original_route']['routes'][0]['duration']} s")
print(f"Distance: {route['original_route']['routes'][0]['distance']} m")

In [ ]:
s = StringIO(
    """
    approach,distance,duration
    current,27065.3,1813.6
    lane topology,38827.4,2713.5
    X-Graph,39663.8,2783.0
    """)

df = pd.read_csv(s, index_col=0, delimiter=',', skipinitialspace=True)

fig = plt.figure()  # Create matplotlib figure

ax = fig.add_subplot(111)  # Create matplotlib axes
ax2 = ax.twinx()  # Create another axes that shares the same x-axis as ax.

width = 0.4

df.distance.plot(kind='bar', color='#3ad21b', ax=ax, width=width, position=1, rot=0)
df.duration.plot(kind='bar', color='#055AFF', ax=ax2, width=width, position=0)

ax.set_ylabel('distance (m)')
ax.yaxis.label.set_color('#3ad21b')
ax2.set_ylabel('duration (s)')
ax2.yaxis.label.set_color('#055AFF')

plt.savefig("/Users/nitrotube/Desktop/durations.png", bbox_inches='tight', dpi=400)
plt.show()

Calculate average straight length (ASL)

In [ ]:
# order = order_lane[1:-1]
order = order_normal[1:-1]
# order = shortened_x_graph_order

straights = []
previous_lanelet = None
current_straight = 0

for lanelet in order:
    if previous_lanelet is None:
        previous_lanelet = lanelet
        continue

    if (previous_lanelet.segment.id, lanelet.segment.id) in rp.maneuvers:
        maneuver = rp.maneuvers[(previous_lanelet.segment.id, lanelet.segment.id)]

        if maneuver.modifier == ManeuverModifier.Straight:
            if current_straight == 0:
                current_straight = previous_lanelet.segment.get_length() + lanelet.segment.get_length()
            else:
                current_straight += lanelet.get_length()
        else:
            if current_straight > 0:
                straights.append(current_straight)
                current_straight = 0
    elif current_straight > 0:
        straights.append(current_straight)
        current_straight = 0

    previous_lanelet = lanelet

if current_straight > 0:
    straights.append(current_straight)

average_straight_length = sum(straights) / len(straights)
print(f"Average straight length: {average_straight_length}")
print(len(list(filter(lambda x: x < 150, straights))))

In [ ]:
x = [i for i in range(len(straights))]
y = straights

ax = plt.axes()
ax.tick_params(
    axis='x',
    which='both',
    bottom=False,
    top=False,
    labelbottom=False)
ax.set_ylabel('length (m)')
ax.set_xlabel('strings')

plt.scatter(x, y, color="#FA05FF")
plt.savefig("/Users/nitrotube/Desktop/strings_x.png", bbox_inches='tight', dpi=400)
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
x_labels = ["<150", "<500", ">500"]
ys = [3273, 1990, 3056]

# add proper binning
ax.bar(x_labels, ys, color=["#3ad21b", "#055AFF", "#FA05FF", "#FFE605"])
plt.xticks(rotation=15)
ax.set_ylabel('number of strings')
ax.set_xlabel('strings length (m)')
plt.savefig("/Users/nitrotube/Desktop/lss.png", bbox_inches='tight', dpi=400)
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
heuristics = ["current", "lane topology", "X-Graph"]
ys = [340, 531, 571.6]
ax.bar(heuristics, ys, color=["#3ad21b", "#055AFF", "#FA05FF"])
ax.set_ylabel('average straight length (s)')
ax.set_xlabel('approach')
plt.savefig("/Users/nitrotube/Desktop/asl.png", bbox_inches='tight', dpi=400)
plt.show()

In [ ]:
optimiser = XGraphOptimizer(nodes=x_nodes,
                            disjunctions=disjunctions,
                            matrix=matrix,
                            local_search_metaheuristic=LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH,
                            first_solution_strategy=FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION,
                            max_optimisation_duration=400,
                            straight_non_straight_maneuver_penalty=120,
                            non_straight_straight_maneuver_penalty=0)
optimal_order, optimization_history = optimiser.optimise()
print(f"Optimisation history: {optimization_history}")
optimal_order = optimal_order[1:-1]

Show X-Graph route on map

In [ ]:
route_connections = []

for connection in optimal_order:
    route_connections.append(RouteConnection(connection.lanelet_from,
                                             connection.lanelet_to,
                                             0,
                                             2))

# Convert stuff to geojson
rp_json = rp.to_json()
connections_json = [connection.to_json() for connection in route_connections]
geojson = FeatureCollection(features=rp_json + connections_json)

# Show everything on map
color_breaks = [0, 1, 2]
color_stops = create_color_stops(color_breaks, colors=["#3ad21b", "#ff0505", "#d742f5"])
viz = LinestringViz(data=geojson,
                    color_property="type",
                    color_stops=color_stops,
                    line_width_default='2',
                    center=(9.092, 48.731),
                    zoom=18,
                    style=satellite_style)
viz.show()